## Example: Leaching of cement with water or clay pore solution 

**Authors: G. Dan Miron**

## Import python packages

In [ ]:
import xgems as xg
import numpy as np
import pandas as pd

## Initalize an xgems chemical engine

In [ ]:
xg_system_filename='gems_files/PC-dat.lst'
xgEngine = xg.ChemicalEngine(xg_system_filename)

## Setting Up Containers for Input and Output

- Rather than defining all indexes manually, it's more efficient to maintain lists of phases and elements. *(Tip: these lists can be copied from GEM-Selektor and edited easily in a text editor.)*  
- You can also use the pre-prepared lists provided in the help folder, such as `elements_phases.txt`.  
- Notice that we work with two types of containers:  
  1. `element_symbols` is a simple array (a list of symbols).  
  2. `phase_names` is a more complex structure called a dictionary — a collection of labeled items, where each item has a key (name) and an associated value.  
- Dictionaries are useful for handling phase name aliases and grouping, and they allow us to build more sophisticated data structures if needed.  
- We can apply the same approach to create dictionaries mapping phases and elements to their respective indexes. This eliminates the need to define each index individually.


In [ ]:
# lists of elements and phases
element_symbols = ['Al', 'C', 'Ca', 'Cl', 'Fe', 'H', 'K', 'Mg', 'Na', 'O', 'S', 'Si', 'Zz']
phase_names = {
    'CSHQ': 'CSH',
    'ettringite': 'ettrignite',
    'SO4_CO3_AFt': 'ettrignite',
    'CO3_SO4_AFt': 'ettrignite',
    'thaumasite': 'thaumasite',
    'SO4_OH_AFm': 'monosulphate',
    'OH_SO4_AFm': 'monosulphate',
    'C4AcH11': 'monocarbonate',
    'OH-hydrotalcite': 'hydrotalcite',
    'Kuzels': 'Kuzel_s',
    'Friedels': 'Friedel_s',
    'straetlingite': 'straetlingite',
    'Calcite': 'calcite',
    'MSH': 'MSH',
    'Gypsum': 'gypsum',
    'Portlandite': 'portlandite',
    'Natrolite': 'zeolites',
    'ZeoliteX': 'zeolites',
    'ZeoliteY': 'zeolites',
    'ZeoliteP': 'zeolites',
    'Chabazite': 'zeolites',
    'C3(AF)S0.84H': 'hydrogarnet',
    'C3FS1.34H3.32': 'hydrogarnet',
    'Ferrihydrite-mc': 'ferryhidrite',
    'Al(OH)3mic': 'Al(OH)3(mic)',
    'aq_gen': 'aqueous',
}

In [ ]:
# A value in a dictionary is accessed using its key, similar to how an index is used to access an element in a normal list.
phase_names['CSHQ']

In [ ]:
# Example for Aluminum (Al), but ultimately, we want to have a dictionary containing all elements.
element_indexes = {'Al': xgEngine.indexElement('Al')}
element_indexes

In [ ]:
# Creating the dictionary manually would require a lot of typing.  
# Instead, we can use a loop to automate the process.  
# By applying the pattern shown above and adding a `for` loop (in Python), we can efficiently build the dictionary.  
# (Tip: ChatGPT can be a great help with generating such code.)

# element_indexes = {symbol: xgEngine.indexElement(symbol)}

element_indexes = {symbol: xgEngine.indexElement(symbol) for symbol in element_symbols}
element_indexes

In [ ]:
# we do the same thing for the phases
phase_indexes = {name: xgEngine.indexPhase(name) for name in phase_names}

In [ ]:
# now we can define what we want as output
output_aqueous_elements = ['Ca', 'Si', 'S', 'C']
output_properties = ['cycle', 'pH', 'gems_code', 'Ca/Si in CSH']

# for phases we keep the same as we have in phase_names - this is a trick to get unique values 
# dict.fromkeys() creates a dictionary with the values as keys (which are unique), preserving their first occurrence order.
# Converting back to a list gives you a unique, ordered list.
output_phases = list(dict.fromkeys(phase_names.values()))

#output_phases

# or we can just use phase_names.values() 

In [ ]:
# initalize the output containers, also dictionaries
# imagine these like an empty table with column names and values in rows

aqueous_table = {element: [] for element in output_aqueous_elements}

# Since we want the group names rather than individual phases,  
# we loop through the dictionary values instead of the keys.
solids_volume_table = {phase: [] for phase in phase_names.values()}

properties_table = {prop: [] for prop in output_properties}
properties_table
solids_volume_table

In [ ]:
# Finally, we want to create a dictionary where the keys are group names (outputs),  
# and the values are lists of indexes for the phases that belong to each group. (i.e., more phases belong to ettringite)

grouped_output_phase_indexes = {}
for phase, name in phase_names.items():
    grouped_output_phase_indexes.setdefault(name, []).append(phase_indexes[phase])
grouped_output_phase_indexes

## Defining the leaching solution and script

In [ ]:
# defining the b_ for leaching solution = water
# we copy the system b and empty it, we then add the formula of water
# this we divide with the molar mass * 1000 to have 1 kg of H2O
# we call it b_solution to be able to reuseit
b_solution = xgEngine.elementAmounts().copy()
b_solution[:] = 1e-9
# 1 kg water
b_solution[element_indexes['H']] = 2
b_solution[element_indexes['O']] = 1 #+ 0.001
b_solution = b_solution*1000/18.015

In [ ]:
# help file
# writting the leaching script

P=1e5 # 10 bar = 1e5 Pa
T=20.0+273.15 # Temperatur in Kelvin

# Initial dummy values
aq_elements = xgEngine.elementAmounts().copy()
aq_elements[:] = 1e-20
aq_mass = 0.0

# Equilibration loop
for c in range(1, 10001):
    xgEngine.setColdStart()

    # Replace pore solution with fresh water
    b_recipe_new = xgEngine.elementAmounts().copy()
    b_recipe_new = b_recipe_new - aq_elements
#    if aq_mass < 10:
    b_recipe_new = b_recipe_new + 1.0*b_solution * aq_mass
#    else:
#        b_recipe_new = b_recipe_new + 1.1*b_solution * 10

    code = xgEngine.equilibrate(T, P, b_recipe_new)
    aq_volume = xgEngine.phaseVolume(phase_indexes['aq_gen']) * 1000  # m³ to L
    aq_mass = xgEngine.phaseMass(phase_indexes['aq_gen'])  # kg
    aq_elements = xgEngine.elementAmountsInPhase(phase_indexes['aq_gen'])
    CSH_elements = xgEngine.elementAmountsInPhase(phase_indexes['CSHQ'])

    # properties
    properties_table['cycle'].append(c)
    properties_table['Ca/Si in CSH'].append(CSH_elements[element_indexes['Ca']]/CSH_elements[element_indexes['Si']])
    properties_table['pH'].append(xgEngine.pH())
    properties_table['gems_code'].append(code)

    # Aqueous species concentrations
    for element in output_aqueous_elements:
        index = element_indexes[element]
        aqueous_table[element].append(1000*aq_elements[index] / aq_volume)

    # Solid phase volumes
    for phase, ndx_list in grouped_output_phase_indexes.items():
        vol = sum(xgEngine.phaseVolume(ndx) for ndx in ndx_list)
        solids_volume_table[phase].append(vol * 1e6)  # m³ to cm³
    # This causes the printed line to be overwritten on each loop iteration, instead of printing a new line each time. 
    print('\rcalculate for cycle', c, 'code', code, end='')    

In [ ]:
# simple plot
import matplotlib.pyplot as plt

plt.plot(properties_table['cycle'], properties_table['pH'], label='pH', color='black', linestyle='--')
plt.legend()
plt.show()

In [ ]:
# plot the results
# ask chat gpt: how to save the figure
import matplotlib.pyplot as plt
import numpy as np

plt.rcParams.update({'font.size': 15})

# Create plot
fig, ax1 = plt.subplots(figsize=(7, 5))

for elem in output_aqueous_elements:
    ax1.plot(properties_table['cycle'], aqueous_table[elem], label=elem)

ax1.set_xlabel('Cycle')
ax1.set_ylabel('total conc. in solution (mmol/l)')
ax1.set_title('Leaching of OPC with water at 20 °C')
ax1.grid(True)
ax1.set_xscale('log')

ax2 = ax1.twinx()
ax2.plot(properties_table['cycle'], properties_table['pH'], label='pH', color='black', linestyle='--')
ax2.set_ylabel('pH')

lines1, labels1 = ax1.get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
ax1.legend(lines1 + lines2, labels1 + labels2, loc='upper right')

plt.tight_layout()
plt.show()

In [ ]:
# Assume solid_data_volumes is a dictionary filled with lists of equal length
# Extract the x-axis (cycle) and the y-values (each mineral phase)
x = properties_table['cycle']

# Y-values in correct order
y_values = [solids_volume_table[phase] for phase in solids_volume_table]

# Labels for legend
labels = [phase for phase in solids_volume_table]  # You can make this prettier manually if needed

# Color palette (optional): provide your own or use matplotlib defaults
colors = plt.cm.tab20c(range(len(solids_volume_table)))  # or a custom list

# Optional hatches, !!! lenght the same as output phases
hatches = ['/', '\\', '|', '-', '+', 'x', 'o', 'O', '.', '*', '//', 'xx', '\\\\', '||', '++', 'oo', '**', '..']

# Create the stacked area plot
fig, ax = plt.subplots(figsize=(12, 6))
stacks = ax.stackplot(x, *y_values, labels=labels, colors=colors, alpha=0.8)

# Apply hatching
for stack, hatch in zip(stacks, hatches):
    stack.set_hatch(hatch)

# Axis labels and limits
ax.set_xlabel('Cycle')
ax.set_ylabel('Phase volume (cm$^3$/100 g cement)')
ax.set_title('Solid Phase Volume Evolution During Leaching')
#ax.set_xlim(min(x), max(x))
#ax.set_ylim(0, max([sum(vals) for vals in zip(*y_values)]) * 1.1)
ax.set_ylim(0, 100)
ax.set_xscale('log')

# Create a second y-axis for pH
ax2 = ax.twinx()
ax2.plot(x, properties_table['Ca/Si in CSH'], label='Ca/Si in CSH', color='black', linestyle='--')
ax2.set_ylabel('Ca/Si in CSH')

# Combine legends from both axes
lines1, labels1 = ax.get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
ax.legend(lines1 + lines2, labels1 + labels2, loc='upper right',  bbox_to_anchor=(1.35, 1.0), fontsize=10)

plt.tight_layout()
plt.show()

## Leaching with clay pore solution

In [ ]:
##write results to file, ask chetgpt how to 

In [ ]:
import pandas as pd

# Convert dicts to DataFrames
df_properties = pd.DataFrame(properties_table)
df_aqueous = pd.DataFrame(aqueous_table)
df_solids = pd.DataFrame(solids_volume_table)

# Concatenate them side-by-side
combined_df = pd.concat([df_properties, df_aqueous, df_solids], axis=1)

# Write to CSV
combined_df.to_csv('combined_data.csv', index=False)

print("Combined data written to combined_data.csv")